In [1]:
from __future__ import division, print_function, absolute_import

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import autoencoder as ae
#import classification as cf
import autoencoder_depart as aed
import depart_classification as dcf
from input import Bearing_dataset

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
learning_rate = 0.01
X = tf.placeholder(dtype = tf.float32, shape = [None, ae.num_input], name = 'layer_1_input')
#layer_2_X = tf.placeholder(dtype = tf.float32, shape = [None, ae.num_hidden_1], name = 'layer_2_input')
#layer_3_X = tf.placeholder(dtype = tf.float32, shape = [None, ae.num_hidden_2], name = 'layer_3_input')
is_training = tf.placeholder(dtype = tf.bool, name = "traing")
layer_1_para = aed.layer_para(X, aed.scope_name[0], is_training, 2.6e-3)
layer_2_X = aed.Encoder(X, aed.scope_name[0], is_training)
layer_2_para = aed.layer_para(layer_2_X, aed.scope_name[1], is_training, 8e-3)
layer_3_X = aed.Encoder(layer_2_X, aed.scope_name[1], is_training)
layer_3_para = aed.layer_para(layer_3_X, aed.scope_name[2], is_training, 6e-3)

def run_autoencoder(session, loss_val, Xd, predict,
                    drop_prob = 0.4,
                    num_steps = 40, batch_size=256, print_every = 100,
                    training = None, plot_losses = False):
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

    train_indices = np.arange(Xd.shape[0])
    np.random.shuffle(train_indices)

    variables = [loss_val, training, extra_update_ops]
   
    iter_cnt = 0
    correct = 0
    losses = []
    for i in range(num_steps):
        start_idx = (i * batch_size)%Xd.shape[0]
        idx = train_indices[start_idx: start_idx + batch_size]

        feed_dict = {X: Xd[idx, :], is_training: training is not None}
        actual_batch_size = Xd[idx].shape[0]
        loss,  _, _ = session.run(variables, feed_dict = feed_dict)
        losses.append(loss * actual_batch_size)
        #correct += np.sum(corr)

        if (iter_cnt%print_every) == 0:
            print("Iteration{0}: with minibatch training loss = {1:.3g}".format(iter_cnt, loss))
        iter_cnt += 1
    total_loss = np.sum(losses)/Xd.shape[0]
    print("Overall loss = {0:.3g} ".format(total_loss))
    if plot_losses:
        plt.plot(losses)
        plt.grid(True)
        plt.title('Loss')
        plt.xlabel("minibatch number")
        plt.ylabel('minibatch loss')
        plt.show()
    return total_loss



In [4]:
train_data = ae.mnist.train.images
train_labels = np.asarray(ae.mnist.train.labels, dtype=np.int32)
eval_data = ae.mnist.test.images
eval_labels = np.asarray(ae.mnist.test.labels, dtype=np.int32)
init = tf.global_variables_initializer()
saver = tf.train.Saver()
b_train_data = Bearing_dataset.train_data
b_train_labels = Bearing_dataset.train_label
b_train_labels = tf.one_hot(b_train_labels, 10)
b_eval_labels = Bearing_dataset.eval_label
b_eval_labels = tf.one_hot(b_eval_labels, 10)
b_eval_data = Bearing_dataset.eval_data


In [5]:
print(b_train_labels)
print(b_train_data.shape)

Tensor("one_hot:0", shape=(800, 10), dtype=float32)
(800, 1200)


In [6]:
with tf.Session() as sess:
    sess.run(init)
    print("Pretrain layer_1:")
    run_autoencoder(sess, layer_1_para['loss'], b_train_data, layer_1_para['y_pred'],
                    1, 1000, 512, 50, 
                    layer_1_para['optimizer'], False)
    print("Pretrain layer_2:")
    run_autoencoder(sess, layer_2_para['loss'], b_train_data, layer_2_para['y_pred'],
                   1, 350, 512, 50,
                   layer_2_para['optimizer'], False)
    print("Pretrain layer_3:")
    run_autoencoder(sess, layer_3_para['loss'], b_train_data, layer_3_para['y_pred'],
                    1, 250, 512, 50,
                    layer_3_para['optimizer'], False)
    print("train")
    dcf.run_model(sess, dcf.out, dcf.loss, b_train_data, b_train_labels.eval(), 0.55,
                 1500, 512, 100,
                 dcf.train_step, False)
    print("eval")
    dcf.run_model(sess, dcf.out, dcf.loss, b_eval_data, b_eval_labels.eval(), 1, 1, 512)
    
    
    save_path = saver.save(sess, "/tmp/model.ckpt")

Pretrain layer_1:
Iteration0: with minibatch training loss = 1.29
Iteration50: with minibatch training loss = 0.678
Iteration100: with minibatch training loss = 0.562
Iteration150: with minibatch training loss = 0.52
Iteration200: with minibatch training loss = 0.493
Iteration250: with minibatch training loss = 0.475
Iteration300: with minibatch training loss = 0.461
Iteration350: with minibatch training loss = 0.451
Iteration400: with minibatch training loss = 0.442
Iteration450: with minibatch training loss = 0.436
Iteration500: with minibatch training loss = 0.429
Iteration550: with minibatch training loss = 0.424
Iteration600: with minibatch training loss = 0.42
Iteration650: with minibatch training loss = 0.416
Iteration700: with minibatch training loss = 0.413
Iteration750: with minibatch training loss = 0.41
Iteration800: with minibatch training loss = 0.407
Iteration850: with minibatch training loss = 0.404
Iteration900: with minibatch training loss = 0.402
Iteration950: with m

Epoch 121, Overall loss = 0.285 and accuracy of 0.935
Epoch 122, Overall loss = 0.282 and accuracy of 0.938
Epoch 123, Overall loss = 0.292 and accuracy of 0.934
Epoch 124, Overall loss = 0.279 and accuracy of 0.949
Epoch 125, Overall loss = 0.275 and accuracy of 0.938
Epoch 126, Overall loss = 0.303 and accuracy of 0.932
Epoch 127, Overall loss = 0.263 and accuracy of 0.941
Epoch 128, Overall loss = 0.262 and accuracy of 0.939
Epoch 129, Overall loss = 0.267 and accuracy of 0.949
Epoch 130, Overall loss = 0.273 and accuracy of 0.948
Epoch 131, Overall loss = 0.283 and accuracy of 0.935
Epoch 132, Overall loss = 0.255 and accuracy of 0.958
Epoch 133, Overall loss = 0.26 and accuracy of 0.941
Epoch 134, Overall loss = 0.244 and accuracy of 0.958
Epoch 135, Overall loss = 0.257 and accuracy of 0.945
Epoch 136, Overall loss = 0.281 and accuracy of 0.931
Epoch 137, Overall loss = 0.255 and accuracy of 0.935
Epoch 138, Overall loss = 0.268 and accuracy of 0.939
Epoch 139, Overall loss = 0.2

Epoch 271, Overall loss = 0.0873 and accuracy of 0.991
Epoch 272, Overall loss = 0.113 and accuracy of 0.984
Epoch 273, Overall loss = 0.113 and accuracy of 0.986
Epoch 274, Overall loss = 0.114 and accuracy of 0.989
Epoch 275, Overall loss = 0.0934 and accuracy of 0.995
Epoch 276, Overall loss = 0.0934 and accuracy of 0.994
Epoch 277, Overall loss = 0.108 and accuracy of 0.985
Epoch 278, Overall loss = 0.083 and accuracy of 0.996
Epoch 279, Overall loss = 0.103 and accuracy of 0.988
Epoch 280, Overall loss = 0.118 and accuracy of 0.98
Epoch 281, Overall loss = 0.0897 and accuracy of 0.993
Epoch 282, Overall loss = 0.0904 and accuracy of 0.991
Epoch 283, Overall loss = 0.115 and accuracy of 0.988
Epoch 284, Overall loss = 0.086 and accuracy of 0.996
Epoch 285, Overall loss = 0.0984 and accuracy of 0.989
Epoch 286, Overall loss = 0.101 and accuracy of 0.99
Epoch 287, Overall loss = 0.0852 and accuracy of 0.993
Epoch 288, Overall loss = 0.0918 and accuracy of 0.989
Epoch 289, Overall los

Epoch 420, Overall loss = 0.0711 and accuracy of 0.991
Epoch 421, Overall loss = 0.0943 and accuracy of 0.986
Epoch 422, Overall loss = 0.0706 and accuracy of 0.995
Epoch 423, Overall loss = 0.0851 and accuracy of 0.993
Epoch 424, Overall loss = 0.0642 and accuracy of 0.99
Epoch 425, Overall loss = 0.075 and accuracy of 0.99
Epoch 426, Overall loss = 0.0673 and accuracy of 0.993
Epoch 427, Overall loss = 0.0858 and accuracy of 0.99
Epoch 428, Overall loss = 0.0767 and accuracy of 0.991
Epoch 429, Overall loss = 0.0883 and accuracy of 0.989
Epoch 430, Overall loss = 0.085 and accuracy of 0.989
Epoch 431, Overall loss = 0.0788 and accuracy of 0.989
Epoch 432, Overall loss = 0.0736 and accuracy of 0.994
Epoch 433, Overall loss = 0.0928 and accuracy of 0.985
Epoch 434, Overall loss = 0.074 and accuracy of 0.991
Epoch 435, Overall loss = 0.0832 and accuracy of 0.993
Epoch 436, Overall loss = 0.0672 and accuracy of 0.993
Epoch 437, Overall loss = 0.0875 and accuracy of 0.988
Epoch 438, Overa

Epoch 570, Overall loss = 0.0647 and accuracy of 0.991
Epoch 571, Overall loss = 0.0644 and accuracy of 0.993
Epoch 572, Overall loss = 0.0758 and accuracy of 0.988
Epoch 573, Overall loss = 0.0799 and accuracy of 0.985
Epoch 574, Overall loss = 0.0762 and accuracy of 0.984
Epoch 575, Overall loss = 0.101 and accuracy of 0.983
Epoch 576, Overall loss = 0.0797 and accuracy of 0.986
Epoch 577, Overall loss = 0.0645 and accuracy of 0.993
Epoch 578, Overall loss = 0.0833 and accuracy of 0.988
Epoch 579, Overall loss = 0.068 and accuracy of 0.993
Epoch 580, Overall loss = 0.0604 and accuracy of 0.993
Epoch 581, Overall loss = 0.084 and accuracy of 0.985
Epoch 582, Overall loss = 0.0627 and accuracy of 0.991
Epoch 583, Overall loss = 0.0687 and accuracy of 0.991
Epoch 584, Overall loss = 0.0722 and accuracy of 0.99
Epoch 585, Overall loss = 0.0815 and accuracy of 0.985
Epoch 586, Overall loss = 0.0893 and accuracy of 0.985
Epoch 587, Overall loss = 0.0603 and accuracy of 0.994
Epoch 588, Ove

...

Epoch 870, Overall loss = 0.0542 and accuracy of 0.996
Epoch 871, Overall loss = 0.052 and accuracy of 0.993
Epoch 872, Overall loss = 0.0534 and accuracy of 0.994
Epoch 873, Overall loss = 0.0498 and accuracy of 0.99
Epoch 874, Overall loss = 0.0519 and accuracy of 0.993
Epoch 875, Overall loss = 0.0567 and accuracy of 0.994
Epoch 876, Overall loss = 0.0445 and accuracy of 0.994
Epoch 877, Overall loss = 0.0672 and accuracy of 0.989
Epoch 878, Overall loss = 0.0514 and accuracy of 0.993
Epoch 879, Overall loss = 0.0349 and accuracy of 1
Epoch 880, Overall loss = 0.0408 and accuracy of 0.999
Epoch 881, Overall loss = 0.0631 and accuracy of 0.99
Epoch 882, Overall loss = 0.0478 and accuracy of 0.995
Epoch 883, Overall loss = 0.0445 and accuracy of 0.995
Epoch 884, Overall loss = 0.0518 and accuracy of 0.996
Epoch 885, Overall loss = 0.062 and accuracy of 0.993
Epoch 886, Overall loss = 0.0413 and accuracy of 0.996
Epoch 887, Overall loss = 0.041 and accuracy of 0.998
Epoch 888, Overall 

Epoch 1020, Overall loss = 0.0752 and accuracy of 0.988
Epoch 1021, Overall loss = 0.0708 and accuracy of 0.989
Epoch 1022, Overall loss = 0.0772 and accuracy of 0.985
Epoch 1023, Overall loss = 0.0622 and accuracy of 0.991
Epoch 1024, Overall loss = 0.0947 and accuracy of 0.985
Epoch 1025, Overall loss = 0.0815 and accuracy of 0.988
Epoch 1026, Overall loss = 0.0669 and accuracy of 0.99
Epoch 1027, Overall loss = 0.07 and accuracy of 0.993
Epoch 1028, Overall loss = 0.0576 and accuracy of 0.994
Epoch 1029, Overall loss = 0.0582 and accuracy of 0.989
Epoch 1030, Overall loss = 0.0625 and accuracy of 0.991
Epoch 1031, Overall loss = 0.07 and accuracy of 0.99
Epoch 1032, Overall loss = 0.0435 and accuracy of 0.996
Epoch 1033, Overall loss = 0.0723 and accuracy of 0.99
Epoch 1034, Overall loss = 0.0556 and accuracy of 0.993
Epoch 1035, Overall loss = 0.0904 and accuracy of 0.985
Epoch 1036, Overall loss = 0.0592 and accuracy of 0.989
Epoch 1037, Overall loss = 0.0503 and accuracy of 0.994

Epoch 1164, Overall loss = 0.0516 and accuracy of 0.993
Epoch 1165, Overall loss = 0.0675 and accuracy of 0.986
Epoch 1166, Overall loss = 0.0686 and accuracy of 0.99
Epoch 1167, Overall loss = 0.0494 and accuracy of 0.991
Epoch 1168, Overall loss = 0.0542 and accuracy of 0.989
Epoch 1169, Overall loss = 0.0637 and accuracy of 0.991
Epoch 1170, Overall loss = 0.0651 and accuracy of 0.99
Epoch 1171, Overall loss = 0.0453 and accuracy of 0.995
Epoch 1172, Overall loss = 0.0509 and accuracy of 0.991
Epoch 1173, Overall loss = 0.0564 and accuracy of 0.993
Epoch 1174, Overall loss = 0.0379 and accuracy of 0.996
Epoch 1175, Overall loss = 0.0498 and accuracy of 0.994
Epoch 1176, Overall loss = 0.0399 and accuracy of 0.996
Epoch 1177, Overall loss = 0.0623 and accuracy of 0.991
Epoch 1178, Overall loss = 0.0539 and accuracy of 0.991
Epoch 1179, Overall loss = 0.0649 and accuracy of 0.986
Epoch 1180, Overall loss = 0.0712 and accuracy of 0.989
Epoch 1181, Overall loss = 0.0483 and accuracy of 

Epoch 1308, Overall loss = 0.0597 and accuracy of 0.989
Epoch 1309, Overall loss = 0.0384 and accuracy of 0.999
Epoch 1310, Overall loss = 0.0519 and accuracy of 0.994
Epoch 1311, Overall loss = 0.0528 and accuracy of 0.995
Epoch 1312, Overall loss = 0.0456 and accuracy of 0.995
Epoch 1313, Overall loss = 0.0683 and accuracy of 0.99
Epoch 1314, Overall loss = 0.0394 and accuracy of 0.999
Epoch 1315, Overall loss = 0.0508 and accuracy of 0.99
Epoch 1316, Overall loss = 0.0657 and accuracy of 0.988
Epoch 1317, Overall loss = 0.0422 and accuracy of 0.994
Epoch 1318, Overall loss = 0.0642 and accuracy of 0.991
Epoch 1319, Overall loss = 0.0455 and accuracy of 0.994
Epoch 1320, Overall loss = 0.0598 and accuracy of 0.989
Epoch 1321, Overall loss = 0.0525 and accuracy of 0.995
Epoch 1322, Overall loss = 0.062 and accuracy of 0.99
Epoch 1323, Overall loss = 0.0601 and accuracy of 0.988
Epoch 1324, Overall loss = 0.0573 and accuracy of 0.99
Epoch 1325, Overall loss = 0.0394 and accuracy of 0.9

Epoch 1452, Overall loss = 0.0525 and accuracy of 0.989
Epoch 1453, Overall loss = 0.0565 and accuracy of 0.988
Epoch 1454, Overall loss = 0.0642 and accuracy of 0.986
Epoch 1455, Overall loss = 0.0458 and accuracy of 0.993
Epoch 1456, Overall loss = 0.0659 and accuracy of 0.99
Epoch 1457, Overall loss = 0.0664 and accuracy of 0.985
Epoch 1458, Overall loss = 0.0514 and accuracy of 0.994
Epoch 1459, Overall loss = 0.0959 and accuracy of 0.984
Epoch 1460, Overall loss = 0.076 and accuracy of 0.984
Epoch 1461, Overall loss = 0.0907 and accuracy of 0.98
Epoch 1462, Overall loss = 0.058 and accuracy of 0.99
Epoch 1463, Overall loss = 0.0838 and accuracy of 0.983
Epoch 1464, Overall loss = 0.0503 and accuracy of 0.986
Epoch 1465, Overall loss = 0.0578 and accuracy of 0.986
Epoch 1466, Overall loss = 0.0935 and accuracy of 0.983
Epoch 1467, Overall loss = 0.0394 and accuracy of 0.995
Epoch 1468, Overall loss = 0.0482 and accuracy of 0.991
Epoch 1469, Overall loss = 0.0693 and accuracy of 0.9